In [ ]:
!pip install pypdf
!pip install sentence_transformers
!pip install pinecone-client
!pip install transformers
!pip install langchain

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import pinecone
#Connect to the vector database 
pinecone.init(api_key="c72b02c0-62fa-4ee2-aa36-6c2384fed7e1", environment="gcp-starter")
pinecone.list_indexes()
index = pinecone.Index("ubuntu-ir-jina")

In [4]:
from langchain.document_loaders import PyPDFLoader
#Load documentation dataset 
loader = PyPDFLoader("/content/drive/My Drive/ubuntu-server-guide-2023-10-15.pdf")
pages = loader.load_and_split()

In [ ]:
from transformers import AutoModel
from numpy.linalg import norm
#Load pre-trained model used to create word embeddings 
cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) # trust_remote_code is needed to use the encode method

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#Chunk the documentation dataset according to the specifications below. 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 8000,
    chunk_overlap  = 1000,
    length_function = len,
    add_start_index = True,
)

In [ ]:
#Generate word embeddings of each chunk using the pre-trained model and inserting it into the vector database 
counter = 0
for i in range(len(pages)):
    text_file = open("page_"+ str(i) + ".txt", "w")
    n = text_file.write(pages[i].page_content)
    text_file.close()
    with open("page_"+ str(i) + ".txt") as f:
      info = f.read()
    texts = text_splitter.create_documents([info])
    for j in texts:
      embedding = model.encode([j.page_content])
      index.upsert([(str(counter),embedding[0].tolist(),{"text":j.page_content})])
      print(f'Embedding count: {counter}')
      counter += 1
print(f'Total number of embeddings: {counter}')

In [ ]:
# Query the vector database to retrieve the top k similar embeddings 
query = "How do i ssh into an external server?"

embedding = model.encode([query]).tolist()

index.query(
  vector=embedding,
  top_k=15,
  include_metadata=True
)